In [ ]:
!pip install pymc3==3.11.5

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
!pip install numba==0.55.1

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pymc3 as pm # for uncertainty quantification and model calibration
from scipy.integrate import solve_ivp # to solve ODE system
from scipy import optimize # to solve minimization problem from least-squares fitting
from numba import jit # to accelerate ODE system RHS evaluations
import theano # to control better pymc3 backend and write a wrapper
import theano.tensor as t # for the wrapper to a custom model to pymc3
# Plotting libs
import matplotlib.pyplot as plt
import altair as alt

seed = 12345 # for the sake of reproducibility :)
np.random.seed(seed)

plt.style.use('seaborn-talk') # beautify the plots!

THEANO_FLAGS='optimizer=fast_compile' # A theano trick

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [ ]:
import io
df_covid=pd.read_csv('covid_19_clean_complete.csv', parse_dates=['Date'])
print(df_covid)
df_covid.info()

                 Province/State  Country/Region      Lat     Long       Date  \
0                           NaN     Afghanistan  33.0000  65.0000 2020-01-22   
1                           NaN         Albania  41.1533  20.1683 2020-01-22   
2                           NaN         Algeria  28.0339   1.6596 2020-01-22   
3                           NaN         Andorra  42.5063   1.5218 2020-01-22   
4                           NaN          Angola -11.2027  17.8739 2020-01-22   
...                         ...             ...      ...      ...        ...   
17131                       NaN           Burma  21.9162  95.9560 2020-03-29   
17132                  Anguilla  United Kingdom  18.2206 -63.0686 2020-03-29   
17133    British Virgin Islands  United Kingdom  18.4207 -64.6400 2020-03-29   
17134  Turks and Caicos Islands  United Kingdom  21.6940 -71.7979 2020-03-29   
17135                       NaN      MS Zaandam   0.0000   0.0000 2020-03-29   

       Confirmed  Deaths  Recovered  
0

In [ ]:
def seirdq_model(t, X, alpha=1/5, beta=1, gamma=0, omega=0, zeta=1/15, delta=0.02):
    """
    A modified SEIRD model in order to take into account quarantine.
    """
    S, E, I, R, D = X
    S_prime = - beta * S * I - gamma * E * S - omega * S
    E_prime = beta * S * I - alpha * E + gamma * E * S - omega * E
    I_prime = alpha * E - zeta * I - delta * I - omega * I
    R_prime = zeta * I + omega * (S + E + I)
    D_prime = delta * I
    return S_prime, E_prime, I_prime, R_prime, D_prime

In [ ]:
def seirdq_ode_solver(y0, t_span, t_eval, beta=1, gamma=0, delta=0.02, omega=0, alpha=1/4, zeta=1/14):
    solution_ODE = solve_ivp(
        fun=lambda t, y: seirdq_model(t, y, alpha=alpha, beta=beta, gamma=gamma, omega=omega, zeta=zeta, delta=delta), 
        t_span=t_span, 
        y0=y0,
        t_eval=t_eval,
        method='LSODA'
    )
    
    return solution_ODE

In [ ]:
solution_ODE_predict_seir = seir_ode_solver(y0_seir, (t0, tf), time_range, beta_fitted_seir, gamma_fitted_seir, alpha_fitted,                                                                                                                 zeta_fitted)  # SEIR
t_computed_predict_seir, y_computed_predict_seir = solution_ODE_predict_seir.t, solution_ODE_predict_seir.y
S_predict_seir, E_predict_seir, I_predict_seir, R_predict_seir = y_computed_predict_seir

IndentationError: unexpected indent (<ipython-input-7-46c65258f128>, line 3)

In [ ]:
seirdq_ode_solver(y0, t_span, t_eval, 1, 0, 0.02,0, 1/4,1/14)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=dc57ba6e-be15-44f8-a950-3572ffb61812' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>